In [ ]:
pip install tensorflow pandas tensorflow scikit-learn geopy transformers sentencepiece

In [ ]:
# Step 1: Import necessary libraries
import pandas as pd
from geopy.geocoders import Nominatim
from transformers import XLNetTokenizer, XLNetForSequenceClassification
from sklearn.metrics.pairwise import cosine_similarity

# Load your dataset
df = pd.read_csv('zomato.csv',encoding='latin1')

# Preprocess the dataset
# Handle missing values
df.dropna(inplace=True) 


In [ ]:
def preprocess_user_input(user_input):
    # Preprocess user input to extract relevant information
    # Example implementation:
    location = extract_location(user_input)
    delivery_preference = extract_delivery_preference(user_input)
    cravings = extract_cravings(user_input)
    return location, delivery_preference, cravings

def extract_location(user_input):
    # Extract location from user input
    # Example implementation:
    # Assuming location is provided directly in the input
    return user_input.get('Location', '')

def extract_delivery_preference(user_input):
    # Extract delivery preference from user input
    # Example implementation:
    return user_input.get('Deliver', 'No')

def extract_cravings(user_input):
    # Extract cravings from user input
    # Example implementation:
    return user_input.get('Any cravings', '')


In [ ]:
# Load XLNet model and tokenizer
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased')

In [ ]:
import numpy as np

# Assume you have loaded the XLNet model and tokenizer earlier

# Function to tokenize text using XLNet tokenizer
def tokenize_text(text):
    return tokenizer.encode_plus(text, add_special_tokens=True, return_tensors='pt', max_length=512, truncation=True)

# Function to calculate cosine similarity between two vectors
def cosine_similarity_vector(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

# Function to generate recommendations based on user input and restaurant dataset
def generate_recommendations(user_input, df):
    location, _, cravings = preprocess_user_input(user_input)
    
    # Filter restaurants based on location
    filtered_df = df[df['City'] == location]
    
    # Tokenize user cravings
    tokenized_user_cravings = tokenize_text(cravings)['input_ids']
    
    # Calculate XLNet embeddings for restaurant cuisines
    cuisine_embeddings = []
    for cuisine in filtered_df['Cuisines']:
        tokenized_cuisine = tokenize_text(cuisine)['input_ids']
        with torch.no_grad():
            outputs = model(input_ids=tokenized_cuisine)
            cuisine_embedding = outputs.last_hidden_state.mean(dim=1).numpy()
            cuisine_embeddings.append(cuisine_embedding)
    
    # Calculate XLNet embedding for user cravings
    with torch.no_grad():
        outputs = model(input_ids=tokenized_user_cravings)
        user_cravings_embedding = outputs.last_hidden_state.mean(dim=1).numpy()
    
    # Calculate cosine similarity between user cravings and restaurant cuisines
    similarities = [cosine_similarity_vector(user_cravings_embedding, ce) for ce in cuisine_embeddings]
    
    # Sort restaurants based on cosine similarity
    filtered_df['Similarity'] = similarities
    filtered_df = filtered_df.sort_values(by='Similarity', ascending=False)
    
    # Return top 1-5 recommended restaurants
    return filtered_df.head(5)['Restaurant Name']

# Example usage:
user_input = {
    'Location': 'Makati City',
    'Deliver': 'No',
    'Any cravings': 'buffet cuisines'
}
recommendations = generate_recommendations(user_input, df)
print(recommendations)